In [5]:
import json
import numpy as np
import os
from pathlib import Path

pert_tx_path = "/home/hice1/stadik3/OpenGERT/path_gain_results/tx_30.538193250692643_-303.43746641653684/"
# Open and read the JSON file
with open(os.path.join(pert_tx_path, "config.json"), 'r') as file:
    config_data = json.load(file)

def process_path_gain_results(
    path_gain_list, grid_origin, cm_cell_size, tx_xy, tx_dir
    ):
    """Process and plot path gain results."""
    # Stack the path gain arrays
    stacked_path_gain = np.stack(path_gain_list, axis=0)
    
    # Create a mask for broken links (values equal to 0.0)
    epsilon = 0.0
    broken_links_mask = (stacked_path_gain == epsilon)
    
    # Count frequency of broken links at each position
    broken_links_frequency = np.sum(broken_links_mask, axis=0)
    
    # Create mask for locations with fewer than maximum broken links
    max_broken_links = np.max(broken_links_frequency)
    valid_locations_mask = (broken_links_frequency < max_broken_links)
    
    # Replace broken links with nan for statistics calculation
    path_gain_masked = stacked_path_gain.copy()
    path_gain_masked[broken_links_mask] = np.nan
    
    # Convert to dB, ignoring nan values
    path_gain_array_db = 10 * np.log10(path_gain_masked)
    
    # Calculate statistics (nan values will be automatically excluded)
    path_gain_mean_db = np.nanmean(path_gain_array_db, axis=0)
    path_gain_std_db = np.nanstd(path_gain_array_db, axis=0)
    
    # Apply valid locations mask to statistics arrays
    path_gain_mean_db_masked = np.where(valid_locations_mask, path_gain_mean_db, np.nan)
    path_gain_std_db_masked = np.where(valid_locations_mask, path_gain_std_db, np.nan)
    broken_links_frequency_masked = np.where(valid_locations_mask, broken_links_frequency, np.nan)

    # Get coverage map extent for plotting
    extent = (
        grid_origin[0] - cm_cell_size[0] * path_gain_mean_db.shape[1] / 2,
        grid_origin[0] + cm_cell_size[0] * path_gain_mean_db.shape[1] / 2,
        grid_origin[1] - cm_cell_size[1] * path_gain_mean_db.shape[0] / 2,
        grid_origin[1] + cm_cell_size[1] * path_gain_mean_db.shape[0] / 2
    )
    
    # Plot mean path gain
    plt.figure(figsize=(8, 6))
    plt.imshow(
        path_gain_mean_db_masked,
        extent=extent,
        origin='lower',
        cmap='hot',
        aspect='auto')
    plt.colorbar(label='Mean Path Gain (dB)')
    material = config_data["sim_material_perturbation"]
    height = config_data["sim_building_height_perturbation"]
    position = config_data["sim_building_position_perturbation"]
    plt.title(f"Mean Path Gain for TX at {tx_xy}\n(Excluding Max Broken Links: {max_broken_links})\nPerturb Material:{material}, Height: {height}, Location: {position}")
    plt.xlabel('X position (m)')
    plt.ylabel('Y position (m)')
    mean_fig_filename = os.path.join(
        tx_dir, f"tx_{tx_x_formatted}_{tx_y_formatted}_path_gain_mean.png")
    plt.savefig(mean_fig_filename)
    plt.close()
    summarize_array_statistics(path_gain_mean_db_masked, tx_xy, extent, filename=mean_fig_filename[:-4] + "_stats" + ".png")
    
    # Plot standard deviation of path gain
    plt.figure(figsize=(8, 6))
    plt.imshow(
        path_gain_std_db_masked,
        extent=extent,
        origin='lower',
        cmap='hot',
        aspect='auto')
    plt.colorbar(label='Std Dev of dB Path Gain')
    plt.title(f"Std Dev of Path Gain for TX at {tx_xy}\n(Excluding Max Broken Links: {max_broken_links})\nPerturb Material:{material}, Height: {height}, Location: {position}")
    plt.xlabel('X position (m)')
    plt.ylabel('Y position (m)')
    std_fig_filename = os.path.join(
        tx_dir, f"tx_{tx_x_formatted}_{tx_y_formatted}_path_gain_std.png")
    plt.savefig(std_fig_filename)
    plt.close()
    summarize_array_statistics(path_gain_std_db_masked, tx_xy, extent, filename=std_fig_filename[:-4] + "_stats" + ".png")
    
    # Plot broken links frequency
    plt.figure(figsize=(8, 6))
    plt.imshow(
        broken_links_frequency_masked,
        extent=extent,
        origin='lower',
        cmap='YlOrRd',  # Different colormap to distinguish from other plots
        aspect='auto')
    plt.colorbar(label='Number of Broken Links')
    plt.title(f"Broken Links Frequency for TX at {tx_xy}\n(Excluding Max Value: {max_broken_links})\nPerturb Material:{material}, Height: {height}, Location: {position}")
    plt.xlabel('X position (m)')
    plt.ylabel('Y position (m)')
    freq_fig_filename = os.path.join(
        tx_dir, f"tx_{tx_x_formatted}_{tx_y_formatted}_broken_links_freq.png")
    plt.savefig(freq_fig_filename)
    plt.close()
    summarize_array_statistics(broken_links_frequency_masked, tx_xy, extent, filename=freq_fig_filename[:-4] + "_stats" + ".png")
    
    # Clean up
    del stacked_path_gain
    del path_gain_masked
    del path_gain_array_db
    del path_gain_mean_db
    del path_gain_std_db
    del broken_links_mask
    del broken_links_frequency
    del path_gain_list
    gc.collect()

def process_channel_stats_results(
    stat_list, stat_name, grid_origin, cm_cell_size, tx_xy, tx_dir
    ):
    """Process and plot channel statistics results."""
    # Compute statistics and create plots
    stat_stck = np.stack(stat_list, axis=0)
    
    # Create masks for invalid data
    all_nan_mask = np.all(np.isnan(stat_stck), axis=0)
    all_zero_mask = np.all(stat_stck == 0.0, axis=0)
    valid_locations_mask = ~(all_nan_mask | all_zero_mask)  # Combine both masks
    
    # Compute statistics
    stat_mean = np.mean(stat_stck, axis=0)
    stat_std = np.std(stat_stck, axis=0)
    
    # Apply mask to statistics arrays
    stat_mean_masked = np.where(valid_locations_mask, stat_mean, np.nan)
    stat_std_masked = np.where(valid_locations_mask, stat_std, np.nan)

    # Save results
    tx_x_formatted = float(f"{tx_xy[0]:.1f}")
    tx_y_formatted = float(f"{tx_xy[1]:.1f}")
    mean_filename = os.path.join(tx_dir, f"tx_{tx_x_formatted}_{tx_y_formatted}_{stat_name}_mean.npy")
    std_filename = os.path.join(tx_dir, f"tx_{tx_x_formatted}_{tx_y_formatted}_{stat_name}_std.npy")
    np.save(mean_filename, stat_mean_masked)
    np.save(std_filename, stat_std_masked)
    
    # Get coverage map extent for plotting
    extent = (
        grid_origin[0] - cm_cell_size[0] * stat_mean.shape[1] / 2,
        grid_origin[0] + cm_cell_size[0] * stat_mean.shape[1] / 2,
        grid_origin[1] - cm_cell_size[1] * stat_mean.shape[0] / 2,
        grid_origin[1] + cm_cell_size[1] * stat_mean.shape[0] / 2
    )
    
    # Plot mean
    plt.figure(figsize=(8, 6))
    plt.imshow(
        stat_mean_masked,
        extent=extent,
        origin='lower',
        cmap='hot',
        aspect='auto')
    plt.colorbar(label=f'Mean {stat_name.upper()}')
    material = config_data["sim_material_perturbation"]
    height = config_data["sim_building_height_perturbation"]
    position = config_data["sim_building_position_perturbation"]
    plt.title(f"Mean {stat_name.upper()} for TX at {tx_xy}\n(Excluding All-NaN and All-Zero Locations)\nPerturb Material:{material}, Height: {height}, Location: {position}")
    plt.xlabel('X position (m)')
    plt.ylabel('Y position (m)')
    mean_fig_filename = os.path.join(
        tx_dir, f"tx_{tx_x_formatted}_{tx_y_formatted}_{stat_name}_mean.png")
    plt.savefig(mean_fig_filename)
    plt.close()
    summarize_array_statistics(stat_mean_masked, tx_xy, extent, filename=mean_fig_filename[:-4] + "_stats" + ".png", log_scale_y=True)
    
    # Plot standard deviation
    plt.figure(figsize=(8, 6))
    plt.imshow(
        stat_std_masked,
        extent=extent,
        origin='lower',
        cmap='hot',
        aspect='auto')
    plt.colorbar(label=f'Std Dev of {stat_name.upper()}')
    plt.title(f"Std Dev of {stat_name.upper()} for TX at {tx_xy}\n(Excluding All-NaN and All-Zero Locations)\nPerturb Material:{material}, Height: {height}, Location: {position}")
    plt.xlabel('X position (m)')
    plt.ylabel('Y position (m)')
    std_fig_filename = os.path.join(
        tx_dir, f"tx_{tx_x_formatted}_{tx_y_formatted}_{stat_name}_std.png")
    plt.savefig(std_fig_filename)
    plt.close()
    summarize_array_statistics(stat_std_masked, tx_xy, extent, filename=std_fig_filename[:-4] + "_stats" + ".png", log_scale_y=True)
    
    # Clean up
    del stat_stck
    del stat_mean
    del stat_std
    del stat_list
    gc.collect()

def summarize_array_statistics(arr, tx_coordinate, rx_extent, filename="array_statistics.png", arr_name="", log_scale_y=False):
    # Ensure NaNs are ignored in statistics calculations
    mean_val = np.nanmean(arr)
    median_val = np.nanmedian(arr)
    std_dev = np.nanstd(arr)
    min_val = np.nanmin(arr)
    max_val = np.nanmax(arr)

    # Create the receiver coordinates based on rx_extent
    x = np.linspace(rx_extent[0], rx_extent[1], arr.shape[1])
    y = np.linspace(rx_extent[2], rx_extent[3], arr.shape[0])
    rx_x, rx_y = np.meshgrid(x, y)

    # Calculate distances for each point in the grid
    distances = np.sqrt((rx_x - tx_coordinate[0])**2 + (rx_y - tx_coordinate[1])**2)

    # Flatten the arrays and remove NaN values from arr and distances
    arr_flat = arr.flatten()
    distances_flat = distances.flatten()
    valid_mask = ~np.isnan(arr_flat)
    arr_flat = arr_flat[valid_mask]
    distances_flat = distances_flat[valid_mask]

    # If log_scale_y is enabled, replace zero values with a small offset
    if log_scale_y:
        min_nonzero_val = np.min(arr_flat[arr_flat > 0])
        arr_flat = np.where(arr_flat == 0, min_nonzero_val * 0.1, arr_flat)

    # Set up the figure
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    fig.suptitle("Distance-Based Scatter Plot and Histogram of Array Values")

    # Scatter plot of values as a function of distance
    axes[0].scatter(distances_flat, arr_flat, alpha=0.7, edgecolors="black")
    axes[0].set_xlabel("Distance from TX")
    axes[0].set_ylabel(f"{arr_name}")

    # Apply logarithmic scale if specified
    if log_scale_y:
        axes[0].set_yscale("log")  # Set log scale for scatter plot y-axis
        axes[1].set_yscale("log")  # Set log scale for histogram y-axis

    # Histogram of the array values
    axes[1].hist(arr_flat, bins=30, color="skyblue", edgecolor="black")
    axes[1].set_xlabel(f"{arr_name} Value")
    axes[1].set_ylabel("Frequency")

    # Save the figure to file
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Adjust layout to include title
    plt.savefig(filename)
    plt.close(fig)  # Close the figure to free memory

    # Display summary statistics
    stats_df = pd.DataFrame({
        "Statistic": ["Mean", "Median", "Standard Deviation", "Min", "Max"],
        "Value": [mean_val, median_val, std_dev, min_val, max_val]
    })
    print("Summary Statistics:")
    print(stats_df.to_string(index=False))

def load_npy_files(folder_path):
    """
    Load specific .npy files from a folder and organize them into separate lists
    based on their file names.
    
    Args:
        folder_path (str): Path to the folder containing .npy files
        
    Returns:
        tuple: Lists containing path_gain, delay_spread, and mean_excess_delay arrays
    """
    # Convert the folder path to a Path object
    folder = Path(folder_path)
    
    # Initialize lists to store the arrays
    path_gain_list = []
    delay_spread_list = []
    mean_excess_delay_list = []
    
    # Get all .npy files in the folder
    npy_files = list(folder.glob('*.npy'))
    
    # Process each file based on its name
    for file_path in npy_files:
        if file_path.name.endswith('path_gain.npy'):
            path_gain_list.append(np.load(file_path))
        elif file_path.name.endswith('delay_spread.npy'):
            delay_spread_list.append(np.load(file_path))
        elif file_path.name.endswith('mean_excess_delay.npy'):
            mean_excess_delay_list.append(np.load(file_path))
    
    return path_gain_list, delay_spread_list, mean_excess_delay_list


{'scene_name': 'munich', 'use_gpu': True, 'analyze_chan_stats': True, 'batch_size': 70, 'output_dir': 'path_gain_results', 'device': '/device:GPU:0', 'num_perturbations': 50, 'tx_antenna_height': 6, 'sim_material_perturbation': True, 'sim_building_height_perturbation': False, 'sim_building_position_perturbation': False, 'verbose': False}
